In [320]:
import numpy as np
import scipy as sp
import math

In [321]:
def fun(i, j, k):
    return np.sin(i + j + k)

def hilbert(i, j, k):
    return 1 / (1 + i + j + k)

In [322]:
# control by rank
def trsvd(matrix, rank):
    U, S, Vh = np.linalg.svd(matrix, full_matrices=True)
    return U[:,0:rank], S[0:rank], Vh[0:rank, :]

#control by an error
def trsvd_ec(matrix, error):
    U, S, Vh = np.linalg.svd(matrix, full_matrices=True)
    sigmas_revrece_cumsum = np.cumsum(S[::-1])[::-1]
    rank = len(sigmas_revrece_cumsum[sigmas_revrece_cumsum > error])
    print(f"Approximant error: {sigmas_revrece_cumsum[rank]}")
    return U[:,0:rank], S[0:rank], Vh[0:rank, :]


# With ranks
def stHOSVD(A, ranks):
    shape = A.shape
    u = []
    scan_matrix_1 = np.reshape(A, (shape[0], shape[1] * shape[2],))
    matrix_for_cycle = scan_matrix_1
    ranks_for_cycle = (shape[0], shape[1], shape[2])
    
    for i in range(len(shape)):
        #print(matrix_for_cycle.shape)
        U, S, Vh = trsvd(matrix_for_cycle, ranks[i])
        u.append(U)
        # print(Vh)
        # print(np.diag(S) @ Vh)
        matrix_for_cycle = np.reshape(np.transpose(np.diag(S) @ Vh, (1, 0)), (ranks_for_cycle[1], Vh.shape[0] * ranks_for_cycle[2]))
        new_ranks_for_cycle = (ranks_for_cycle[1], ranks_for_cycle[2], Vh.shape[0])
        ranks_for_cycle = new_ranks_for_cycle
        if i == (len(shape) - 1):
            return np.reshape(matrix_for_cycle, ranks), u

# With error
def stHOSVD_ec(A, error):
    shape = A.shape
    u = []
    ranks = []
    scan_matrix_1 = np.reshape(A, (shape[0], shape[1] * shape[2],))
    matrix_for_cycle = scan_matrix_1
    ranks_for_cycle = (shape[0], shape[1], shape[2])
    
    for i in range(len(shape)):
        # print(matrix_for_cycle.shape)
        U, S, Vh = trsvd_ec(matrix_for_cycle, error / np.sqrt(len(shape)))
        u.append(U)
        ranks.append(Vh.shape[0])
        # print(Vh)
        # print(np.diag(S) @ Vh)
        matrix_for_cycle = np.reshape(np.transpose(np.diag(S) @ Vh, (1, 0)), (ranks_for_cycle[1], Vh.shape[0] * ranks_for_cycle[2]))
        new_ranks_for_cycle = (ranks_for_cycle[1], ranks_for_cycle[2], Vh.shape[0])
        ranks_for_cycle = new_ranks_for_cycle
        if i == (len(shape) - 1):
            return np.reshape(matrix_for_cycle, ranks), u


In [323]:
fun = hilbert
n = 100
tensor = np.fromfunction(fun, (n, n, n), dtype=float)
# sprint(tensor)

In [324]:
ranks = (2, 2, 2)
g1, u1 = stHOSVD(tensor, ranks)

In [325]:
tensor_reconstructed_with_ranks = np.einsum('ia,jb,kc,abc->ijk', u1[0], u1[1], u1[2], g1)

In [326]:
print(f"Error for TD with ranks {ranks}: {np.linalg.norm(tensor_reconstructed_with_ranks - tensor)}")

Error for TD with ranks (2, 2, 2): 0.9346942805021775


In [327]:
error = 1e-3
g2, u2 = stHOSVD_ec(tensor, error)

Approximant error: 0.000423425014569482
Approximant error: 0.00040280632620690406
Approximant error: 0.00037999509651471316


In [328]:
tensor_reconstructed_with_ec = np.einsum('ia,jb,kc,abc->ijk', u2[0], u2[1], u2[2], g2)

In [329]:
print(f"Tuker ranks for error {error}: {g.shape}")
print(np.linalg.norm(tensor_reconstructed_with_ec - tensor))

Tuker ranks for error 0.001: (7, 7, 7)
0.0005875636225964203
